In [1]:
import findspark
findspark.init()
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark import *
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("SPARK_ML_SistemaRecomendacion").getOrCreate()

In [4]:
# lines = spark.read.text("ml-latest-small/ratings.csv").rdd
# parts = lines.map(lambda row: row.value.split(";"))
# ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]), rating=float(p[2]), timestamp=int(p[3])))

In [ ]:
# spark.read.option("delimiter", ",").option("quote", '"').option("escape", '"').csv("data/anime.csv", header=True, inferSchema=True)

In [86]:
# CABECERA:
# ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
df_animes = spark.read.csv("data/anime.csv", header=True, sep=",", encoding="UTF-8", inferSchema=True)
#ratingsRDD = df.rdd.map(lambda row: Row(userId=int(row['ID']), animeId=int(row['anime_id']), rating=int(row['rating'])))
#df.printSchema()

# La ordenacion por ID no funciona correctamente, ya que el ID es un string
#df_animes.orderBy(df_animes['ID'].asc()).show()


In [91]:
df_animes.filter(df_animes['ID'] == 37490).show()

+-----+--------------------+-----+--------------------+--------------------+------------------------------------+----+--------+--------------------+---------+---------------+----------+-----------+------+---------------+--------------------+------+----------+-------+---------+--------+---------+-------+-------+-------------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|   ID|                Name|Score|              Genres|        English name|                       Japanese name|Type|Episodes|               Aired|Premiered|      Producers| Licensors|    Studios|Source|       Duration|              Rating|Ranked|Popularity|Members|Favorites|Watching|Completed|On-Hold|Dropped|Plan to Watch|Score-10|Score-9|Score-8|Score-7|Score-6|Score-5|Score-4|Score-3|Score-2|Score-1|
+-----+--------------------+-----+--------------------+--------------------+------------------------------------+----+--------+--------------------+---------+---------------+----------

In [50]:
from pyspark.sql.functions import regexp_replace

In [41]:
df_resultado = df_animes.withColumn("Name", regexp_replace(df_animes["Name"], '^"|"$', ''))

In [40]:
#df_resultado = df_resultado.withColumn("Name", regexp_replace(df_animes["Name"], "['"]+", "'"))

AnalysisException: Resolved attribute(s) Name#2388 missing from ID#2387,Name#2634,Score#2389,Genres#2390,English name#2391,Japanese name#2392,Type#2393,Episodes#2394,Aired#2395,Premiered#2396,Producers#2397,Licensors#2398,Studios#2399,Source#2400,Duration#2401,Rating#2402,Ranked#2403,Popularity#2404,Members#2405,Favorites#2406,Watching#2407,Completed#2408,On-Hold#2409,Dropped#2410,Plan to Watch#2411,Score-10#2412,Score-9#2413,Score-8#2414,Score-7#2415,Score-6#2416,Score-5#2417,Score-4#2418,Score-3#2419,Score-2#2420,Score-1#2421 in operator !Project [ID#2387, regexp_replace(Name#2388, ['"]+, ', 1) AS Name#2671, Score#2389, Genres#2390, English name#2391, Japanese name#2392, Type#2393, Episodes#2394, Aired#2395, Premiered#2396, Producers#2397, Licensors#2398, Studios#2399, Source#2400, Duration#2401, Rating#2402, Ranked#2403, Popularity#2404, Members#2405, Favorites#2406, Watching#2407, Completed#2408, On-Hold#2409, Dropped#2410, ... 11 more fields]. Attribute(s) with the same name appea

In [42]:
df_resultado.show()

+---+--------------------+-----+--------------------+--------------------+------------------------------+-----+--------+--------------------+-----------+--------------------+--------------------+----------------+-----------+---------------+--------------------+------+----------+---------+---------+--------+---------+-------+-------+-------------+--------+--------+--------+--------+--------+-------+-------+-------+-------+-------+
| ID|                Name|Score|              Genres|        English name|                 Japanese name| Type|Episodes|               Aired|  Premiered|           Producers|           Licensors|         Studios|     Source|       Duration|              Rating|Ranked|Popularity|  Members|Favorites|Watching|Completed|On-Hold|Dropped|Plan to Watch|Score-10| Score-9| Score-8| Score-7| Score-6|Score-5|Score-4|Score-3|Score-2|Score-1|
+---+--------------------+-----+--------------------+--------------------+------------------------------+-----+--------+------------

In [44]:
df_resultado.filter(df_animes['ID'] == '2387').select("Name").show(20, False)

+-----------------------------+
|Name                         |
+-----------------------------+
|Ai Monogatari: 9 Love Stories|
+-----------------------------+



In [14]:

df_animes.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- English name: string (nullable = true)
 |-- Japanese name: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Episodes: string (nullable = true)
 |-- Aired: string (nullable = true)
 |-- Premiered: string (nullable = true)
 |-- Producers: string (nullable = true)
 |-- Licensors: string (nullable = true)
 |-- Studios: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Ranked: string (nullable = true)
 |-- Popularity: string (nullable = true)
 |-- Members: double (nullable = true)
 |-- Favorites: integer (nullable = true)
 |-- Watching: integer (nullable = true)
 |-- Completed: integer (nullable = true)
 |-- On-Hold: integer (nullable = true)
 |-- Dropped: integer (nullable = true)
 |-- Plan to Watch: integer (nullable = true)

In [7]:
# Imprime el numero de filas
df_animes.count()

17562

In [8]:
# Convertir la columna ID a int
df_animes = df_animes.withColumn("ID", df_animes["ID"].cast("int"))
df_animes.printSchema()
df_animes.orderBy(df_animes['ID'].asc()).show()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- English name: string (nullable = true)
 |-- Japanese name: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Episodes: string (nullable = true)
 |-- Aired: string (nullable = true)
 |-- Premiered: string (nullable = true)
 |-- Producers: string (nullable = true)
 |-- Licensors: string (nullable = true)
 |-- Studios: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Ranked: string (nullable = true)
 |-- Popularity: string (nullable = true)
 |-- Members: string (nullable = true)
 |-- Favorites: string (nullable = true)
 |-- Watching: string (nullable = true)
 |-- Completed: string (nullable = true)
 |-- On-Hold: string (nullable = true)
 |-- Dropped: string (nullable = true)
 |-- Plan to Watch: string (nullable = true)
 |-- 

# VALORACIONES

## Lectura del csv rating_complete

In [81]:
df_ratings = spark.read.csv("data/rating_complete.csv", header=True, sep=",", encoding="UTF-8", inferSchema=True)
df_ratings = df_ratings.withColumn("user_id", df_ratings["user_id"].cast("int"))
df_ratings = df_ratings.withColumn("anime_id", df_ratings["anime_id"].cast("int"))
df_ratings = df_ratings.withColumn("rating", df_ratings["rating"].cast("int"))
ratingsRDD = df_ratings.rdd.map(lambda row: Row(userId=int(row['user_id']), animeId=int(row['anime_id']), rating=int(row['rating'])))
df_ratings.printSchema()
df_ratings.show()

# Cuantos usuarios hay en las valoraciones
# df_ratings.select('user_id').distinct().count()

root
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- rating: integer (nullable = true)

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      0|     430|     9|
|      0|    1004|     5|
|      0|    3010|     7|
|      0|     570|     7|
|      0|    2762|     9|
|      0|     431|     8|
|      0|     578|    10|
|      0|     433|     6|
|      0|    1571|    10|
|      0|     121|     9|
|      0|     356|     9|
|      0|    1250|     7|
|      0|    2913|     6|
|      0|    1689|     6|
|      0|      68|     6|
|      0|    1829|     7|
|      0|     600|     6|
|      0|    3418|     9|
|      0|     164|     8|
|      0|    1894|     7|
+-------+--------+------+
only showing top 20 rows



In [52]:
# Numero de filas del csv de rating_complete
df_ratings.count()

57633278

## Lectura del csv valoraciones_EP

In [82]:
# No tiene cabecera
df_ratings_ep = spark.read.csv("data/valoraciones_EP.csv", header=False, sep=",", encoding="UTF-8", inferSchema=True)
df_ratings_ep.show()
# Renombrando las columnas
df_ratings_ep = df_ratings_ep.withColumnRenamed("_c0", "user_id")
df_ratings_ep = df_ratings_ep.withColumnRenamed("_c1", "anime_id")
df_ratings_ep = df_ratings_ep.withColumnRenamed("_c2", "rating")
df_ratings_ep.show()

+------+-----+----+
|   _c0|  _c1| _c2|
+------+-----+----+
|666666|10417| 8.0|
|666666| 1358| 7.0|
|666666|13601|10.0|
|666666| 2001| 9.0|
|666666|21877| 8.0|
|666666|28851| 7.0|
|666666|28999| 9.0|
|666666|30276| 9.0|
|666666|31240| 9.0|
|666666|32281| 9.0|
|666666|34134| 7.0|
|666666|35790| 9.0|
|666666|37349| 8.0|
|666666|37403| 5.0|
|666666|37435| 6.0|
|666666|37450| 9.0|
|666666|37520| 9.0|
|666666|37779| 8.0|
|666666|37779| 9.0|
|666666|37982| 8.0|
+------+-----+----+
only showing top 20 rows

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
| 666666|   10417|   8.0|
| 666666|    1358|   7.0|
| 666666|   13601|  10.0|
| 666666|    2001|   9.0|
| 666666|   21877|   8.0|
| 666666|   28851|   7.0|
| 666666|   28999|   9.0|
| 666666|   30276|   9.0|
| 666666|   31240|   9.0|
| 666666|   32281|   9.0|
| 666666|   34134|   7.0|
| 666666|   35790|   9.0|
| 666666|   37349|   8.0|
| 666666|   37403|   5.0|
| 666666|   37435|   6.0|
| 666666|   37450|   9.0|


In [54]:
# Convertir las columnas a int
df_ratings_ep = df_ratings_ep.withColumn("user_id", df_ratings_ep["user_id"].cast("int"))
df_ratings_ep = df_ratings_ep.withColumn("anime_id", df_ratings_ep["anime_id"].cast("int"))
df_ratings_ep = df_ratings_ep.withColumn("rating", df_ratings_ep["rating"].cast("int"))
df_ratings_ep.printSchema()
# Ordenar por anime_id
df_ratings_ep.orderBy(df_ratings_ep['anime_id'].asc()).show()

root
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- rating: integer (nullable = true)

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
| 666666|      47|     9|
| 666666|      72|     9|
| 666666|     523|     9|
| 666666|     543|     8|
| 666666|    1358|     7|
| 666666|    2001|     9|
| 666666|   10417|     8|
| 666666|   13601|    10|
| 666666|   21877|     8|
| 666666|   28851|     7|
| 666666|   28999|     9|
| 666666|   30276|     9|
| 666666|   31240|     9|
| 666666|   32281|     9|
| 666666|   34134|     7|
| 666666|   35790|     9|
| 666666|   37349|     8|
| 666666|   37403|     5|
| 666666|   37435|     6|
| 666666|   37450|     9|
+-------+--------+------+
only showing top 20 rows



In [83]:
# Union de los dos dataframes
df_ratings_completo = df_ratings.union(df_ratings_ep)
df_ratings_completo.show()

# Numero de usuarios en el csv completo
df_ratings_completo.select('user_id').distinct().count()

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      0|     430|   9.0|
|      0|    1004|   5.0|
|      0|    3010|   7.0|
|      0|     570|   7.0|
|      0|    2762|   9.0|
|      0|     431|   8.0|
|      0|     578|  10.0|
|      0|     433|   6.0|
|      0|    1571|  10.0|
|      0|     121|   9.0|
|      0|     356|   9.0|
|      0|    1250|   7.0|
|      0|    2913|   6.0|
|      0|    1689|   6.0|
|      0|      68|   6.0|
|      0|    1829|   7.0|
|      0|     600|   6.0|
|      0|    3418|   9.0|
|      0|     164|   8.0|
|      0|    1894|   7.0|
+-------+--------+------+
only showing top 20 rows



310060

In [3]:
df_animes = spark.read.csv("data/anime.csv", header=True, sep=",", encoding="UTF-8", inferSchema=True)

In [4]:
df_animes.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- English name: string (nullable = true)
 |-- Japanese name: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Episodes: string (nullable = true)
 |-- Aired: string (nullable = true)
 |-- Premiered: string (nullable = true)
 |-- Producers: string (nullable = true)
 |-- Licensors: string (nullable = true)
 |-- Studios: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Ranked: string (nullable = true)
 |-- Popularity: string (nullable = true)
 |-- Members: double (nullable = true)
 |-- Favorites: integer (nullable = true)
 |-- Watching: integer (nullable = true)
 |-- Completed: integer (nullable = true)
 |-- On-Hold: integer (nullable = true)
 |-- Dropped: integer (nullable = true)
 |-- Plan to Watch: integer (nullable = true)

In [5]:
df_animes.count()

17562

In [6]:
df_ratings = spark.read.csv("data/rating_complete.csv", header=True, sep=",", encoding="UTF-8", inferSchema=True)

df_ratings_ep = spark.read.csv("data/valoraciones_EP.csv", header=False, sep=",", encoding="UTF-8", inferSchema=True)
df_ratings_ep = df_ratings_ep.withColumnRenamed("_c0", "user_id")
df_ratings_ep = df_ratings_ep.withColumnRenamed("_c1", "anime_id")
df_ratings_ep = df_ratings_ep.withColumnRenamed("_c2", "rating")

In [7]:
df_ratings_completo = df_ratings.union(df_ratings_ep)

In [103]:
df_ratings_completo.show()

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      0|     430|   9.0|
|      0|    1004|   5.0|
|      0|    3010|   7.0|
|      0|     570|   7.0|
|      0|    2762|   9.0|
|      0|     431|   8.0|
|      0|     578|  10.0|
|      0|     433|   6.0|
|      0|    1571|  10.0|
|      0|     121|   9.0|
|      0|     356|   9.0|
|      0|    1250|   7.0|
|      0|    2913|   6.0|
|      0|    1689|   6.0|
|      0|      68|   6.0|
|      0|    1829|   7.0|
|      0|     600|   6.0|
|      0|    3418|   9.0|
|      0|     164|   8.0|
|      0|    1894|   7.0|
+-------+--------+------+
only showing top 20 rows



# Join con el csv de peliculas para saber qué película qué valoración

In [6]:
# Pregunta por el id de un usuario
id_usuario = input("Introduzca el id del usuario, del que quiere conocer sus valoraciones: ")

# Busca a ese usuario en el dataframe de valoraciones completo
df_ratings_usuario = df_ratings_completo.filter(df_ratings_completo['user_id'] == id_usuario)


In [219]:
df_ratings_usuario.show()

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      0|     430|   9.0|
|      0|    1004|   5.0|
|      0|    3010|   7.0|
|      0|     570|   7.0|
|      0|    2762|   9.0|
|      0|     431|   8.0|
|      0|     578|  10.0|
|      0|     433|   6.0|
|      0|    1571|  10.0|
|      0|     121|   9.0|
|      0|     356|   9.0|
|      0|    1250|   7.0|
|      0|    2913|   6.0|
|      0|    1689|   6.0|
|      0|      68|   6.0|
|      0|    1829|   7.0|
|      0|     600|   6.0|
|      0|    3418|   9.0|
|      0|     164|   8.0|
|      0|    1894|   7.0|
+-------+--------+------+
only showing top 20 rows



In [7]:
df_vistas = df_ratings_usuario.join(df_animes, df_ratings_usuario["anime_id"] == df_animes["ID"], "inner")
df_vistas.show()
df_vistas.count()

+-------+--------+------+----+--------------------+-----+--------------------+--------------------+--------------------------------------+-----+--------+--------------------+-----------+--------------------+--------------------+--------------------+------------+---------------+--------------------+-------+----------+---------+---------+--------+---------+-------+-------+-------------+--------+--------+--------+--------+-------+-------+-------+-------+-------+-------+
|user_id|anime_id|rating|  ID|                Name|Score|              Genres|        English name|                         Japanese name| Type|Episodes|               Aired|  Premiered|           Producers|           Licensors|             Studios|      Source|       Duration|              Rating| Ranked|Popularity|  Members|Favorites|Watching|Completed|On-Hold|Dropped|Plan to Watch|Score-10| Score-9| Score-8| Score-7|Score-6|Score-5|Score-4|Score-3|Score-2|Score-1|
+-------+--------+------+----+--------------------+-----

35

In [ ]:
df_noVistas = df_animes.join(df_vistas, df_animes["ID"] == df_vistas["anime_id"], "left_anti")
df_noVistas.show()
df_noVistas.count()

In [ ]:
df_ratings_completo.filter(df_ratings_completo['anime_id'] == df_animes['ID'])

In [135]:
df_novistas.count()

614635

In [129]:
df_vistas = df_ratings_usuario.join(df_animes, df_ratings_usuario["anime_id"] == df_animes["ID"])
df_vistas.show()

+-------+--------+------+----+--------------------+-----+--------------------+--------------------+--------------------------------------+-----+--------+--------------------+-----------+--------------------+--------------------+--------------------+------------+---------------+--------------------+-------+----------+---------+---------+--------+---------+-------+-------+-------------+--------+--------+--------+--------+-------+-------+-------+-------+-------+-------+
|user_id|anime_id|rating|  ID|                Name|Score|              Genres|        English name|                         Japanese name| Type|Episodes|               Aired|  Premiered|           Producers|           Licensors|             Studios|      Source|       Duration|              Rating| Ranked|Popularity|  Members|Favorites|Watching|Completed|On-Hold|Dropped|Plan to Watch|Score-10| Score-9| Score-8| Score-7|Score-6|Score-5|Score-4|Score-3|Score-2|Score-1|
+-------+--------+------+----+--------------------+-----

In [130]:
df_vistas.count()

35

In [234]:
df_ratings_completo.show()

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      0|     430|   9.0|
|      0|    1004|   5.0|
|      0|    3010|   7.0|
|      0|     570|   7.0|
|      0|    2762|   9.0|
|      0|     431|   8.0|
|      0|     578|  10.0|
|      0|     433|   6.0|
|      0|    1571|  10.0|
|      0|     121|   9.0|
|      0|     356|   9.0|
|      0|    1250|   7.0|
|      0|    2913|   6.0|
|      0|    1689|   6.0|
|      0|      68|   6.0|
|      0|    1829|   7.0|
|      0|     600|   6.0|
|      0|    3418|   9.0|
|      0|     164|   8.0|
|      0|    1894|   7.0|
+-------+--------+------+
only showing top 20 rows



In [7]:
splits = df_ratings_completo.randomSplit([1.0, 2.0])

In [8]:
(training, test) = df_ratings_completo.randomSplit([0.8, 0.2])
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="anime_id", ratingCol="rating")



In [ ]:
model = als.fit(training)

In [ ]:

predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions) 
print("Error numero = " + str(rmse))

In [15]:
ratings = spark.createDataFrame(ratingsRDD)

In [ ]:
(training, test) = ratings.randomSplit([0.8, 0.2])
# Entrenamos el modelo. La estrategia cold start con 'drop' descarata valores NaN en evaluación
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(training)
# Evaluamos el modelo con RMSE
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions) 
print("Root-mean-square error = " + str(rmse))

In [ ]:
# Generamos las 10 mejores recomendaciones para cada usuario
userRecs = model.recommendForAllUsers(10)
# Generamos los top 10 usuarios para cada película
movieRecs = model.recommendForAllItems(10)
# Generar las 10 mejores recomendaciones para un subconjunto de usuarios
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)
# Geenarar la recomendación con el top 10 usuarios para el subconjunto de películas dado
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import split

# Crear una SparkSession
spark = SparkSession.builder.appName("EjemploCSV").getOrCreate()

# Leer el CSV con cabecera y separador ";"
df = spark.read.csv("ml-latest-small/ratings.csv", header=True, sep=";")

# Mostrar el esquema y los primeros registros
df.printSchema()
df.show()

# Dividir la columna en múltiples columnas
df_split = df.withColumn("split_col", split(df["userId,movieId,rating,timestamp"], ","))

# Crear columnas individuales para userId, movieId, rating, timestamp
df_split = df_split.withColumn("userId", df_split["split_col"].getItem(0).cast("int"))
df_split = df_split.withColumn("movieId", df_split["split_col"].getItem(1).cast("int"))
df_split = df_split.withColumn("rating", df_split["split_col"].getItem(2).cast("float"))
df_split = df_split.withColumn("timestamp", df_split["split_col"].getItem(3).cast("int"))

# Crear el RDD a partir de las columnas individuales
ratingsRDD = df_split.rdd.map(lambda row: Row(userId=row['userId'], movieId=row['movieId'], rating=row['rating'], timestamp=row['timestamp']))

# Crear el DataFrame a partir del RDD
ratings = spark.createDataFrame(ratingsRDD)

# Mostrar el esquema y los primeros registros del nuevo DataFrame
ratings.printSchema()
ratings.show()